In [1]:
import pandas as pd
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer

# df=pd.read_csv('C:\\temp\\2020_Jan_items_master.csv')
df=pd.read_csv('C:\\temp\\2020_July_items_master.csv')

Part_Desc='PART_DESC'

df['PART NO_DESC']=df[Part_Desc].str.strip()
df['PART NO_DESC']=df[Part_Desc].str.replace('-',",", regex=True)
df['PART NO_DESC']=df[Part_Desc].str.upper()

df.rename(columns={'Items_': 'items_'},inplace=True)
df.head()
df.describe()


Using TensorFlow backend.


,PART_NO,PART_DESC,items_,PART_NO 12NC,PART NO_DESC
count,1148,1148,1148,1148,1148
unique,1123,1011,44,985,1001
top,77-141-0000040-00,Assy. Wafer E-Chuck/Grounder Driver,other,77-604-04004A0,ASSY. WAFER E-CHUCK/GROUNDER DRIVER
freq,3,8,820,5,8


In [3]:
from sklearn.model_selection import train_test_split
desc=df['PART NO_DESC'].values
item=df['items_'].values
Xd_train, Xd_test, y_train, y_test= train_test_split(desc, item, test_size=0.3)

print(len(desc))

1148


In [4]:
vectorizer=CountVectorizer(min_df=0.0001,   strip_accents='unicode',)
X_train=vectorizer.fit_transform(Xd_train)
X_test=vectorizer.transform(Xd_test)
X_train

<803x1461 sparse matrix of type '<class 'numpy.int64'>'
	with 5422 stored elements in Compressed Sparse Row format>

# Base model

In [5]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
score

C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9014492753623189

In [6]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print (score)

C:\Users\suchung\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9478260869565217


In [7]:
from sklearn.svm import SVC
this_C=1
this_gamma=0.01
clf2=SVC(kernel='linear',C=this_C, gamma=this_gamma).fit(X_train, y_train)
score=clf2.score(X_test, y_test)

print (score)

# print (y_test)
predict=clf2.predict(X_test)

t_result=pd.DataFrame({'y_test':y_test, 'predict':predict})
t_result.to_csv('c:\\temp\\t_result2.csv')

0.9507246376811594


# from sklearn.tree import DecisionTreeClassifier
import pydotplus
clf=DecisionTreeClassifier()
clf.fit(X_train, y_train)
score=clf.score(X_test, y_test)
print (score)


# predict all data to trace back the in accuracy

In [8]:
Xd_all=desc
y_all=item
X_all=vectorizer.transform(Xd_all)
score=clf2.score(X_all, y_all)
print (score)
predict=clf2.predict(X_all)
df['predict']=predict
df.to_csv('c:\\temp\\all_result.csv')
print (predict)

0.9825783972125436
['other' 'other' 'other' ... 'other' 'other' 'other']


## Apply to New Dataset

In [9]:
dft=pd.read_csv('c:\\temp\\blah-testq5.csv')
dft

print (dft.columns)
len(dft['PART_DESC'])
dft
###########**** make sure no empty end-row  *****########## 

Index(['MACHINE_REGION', 'REF_NO', 'CUSTOMER', 'MACHINE_NO',
       'PARTS_REQUEST_REASON_CODE', 'APPLY_DATE', 'Year[APPLY_DATE]',
       'NEED_BY', 'SO_NO', 'SO_LINE', 'PART_NO', 'PART_DESC',
       'NEED_TO_BE_RETURNED', 'SHIPPED_QTY', 'SHIPPING_ORG', 'SUBINVENTORY',
       'DELIVERY_NUM', 'ACTUAL_SHIPPED_DATE', 'APPROVED_DATE_2NDLINE',
       'PRIORITY', 'FILLER', 'APPLICANT', 'PARTS_REQUEST_STATUS',
       'INV_AVAILABLE_DATE', 'INSTALLED_DATE', 'NON_INSTALLED_REASON',
       'NON_INSTALLED_RETURN', 'NON_INSTALL_DAYS', 'INSTALLATION_STATUS',
       'RMA_NO', 'RETURN_DATE', 'FORM_STATUS', 'RETURNED_QTY', 'HMI_SN',
       'HMI_SITE', 'TYPE', 'MACHINE_TYPE', 'DEMO', 'DELIVERY_START',
       'DELIVERY_END', 'INSTALLATION_START', 'INSTALLATION_END', 'CST_DATE',
       'EVALUATION_START', 'EVALUATION_END', 'ACCEPTANCE_DATE',
       'WARRANTY_START', 'WARRANTY_END', 'textbox30', 'SVC_START', 'SVC_END',
       'CURRENCY', 'FULL_AMOUNT', 'REMARK_OF_SVC', 'STATUS_NAME',
       'INSTALLATION_

,MACHINE_REGION,REF_NO,CUSTOMER,MACHINE_NO,PARTS_REQUEST_REASON_CODE,APPLY_DATE,Year[APPLY_DATE],NEED_BY,SO_NO,SO_LINE,...,CURRENCY,FULL_AMOUNT,REMARK_OF_SVC,STATUS_NAME,INSTALLATION_CYCLE_TIME,ACT_CYCLE_TIME,ACT_OVERDUE_DATE,FAB,PART_NO_12,Items_
0,JP,PartsRequest-2020120085,Kioxia,2260,Trouble Shooting,2020/12/13,2020,3.69e+9,84000363,1.1,...,NaN,NaN,NaN,Warranty,45,86,2020/11/22,Y4,77-606-0410017,NaN
1,CN-Shanghai,PartsRequest-2020110021,YMTC,2263,DOA,2020/11/04,2020,3.69e+9,75016501,1.1,...,NaN,NaN,NaN,Evaluation,57,0,2020/12/08,F1,77-604-1105GC2,NaN
2,CN-Shanghai,PartsRequest-2020110021,YMTC,2263,DOA,2020/11/04,2020,3.69e+9,75016501,2.1,...,NaN,NaN,NaN,Evaluation,57,0,2020/12/08,F1,77-604-11057A1,NaN
3,CN-Shanghai,PartsRequest-2020110044,JHICC,12106,PM,2020/11/10,2020,3.69e+9,72000401,19.1,...,CNY,1761132.00,不含13%VAT (PO#CAM1443200685),Out of Warranty_SVC,19,138,2019/03/07,NaN,77-649-0500011,NaN
4,CN-Shanghai,PartsRequest-2020110044,JHICC,12106,PM,2020/11/10,2020,3.69e+9,72000401,20.1,...,CNY,1761132.00,不含13%VAT (PO#CAM1443200685),Out of Warranty_SVC,19,138,2019/03/07,NaN,77-649-0500012,NaN
5,CN-Shanghai,PartsRequest-2020110044,JHICC,12106,PM,2020/11/10,2020,3.69e+9,72000401,21.1,...,CNY,1761132.00,不含13%VAT (PO#CAM1443200685),Out of Warranty_SVC,19,138,2019/03/07,NaN,77-649-0500012,NaN
6,CN-Shanghai,PartsRequest-2020110044,JHICC,12106,PM,2020/11/10,2020,3.69e+9,72000401,22.1,...,CNY,1761132.00,不含13%VAT (PO#CAM1443200685),Out of Warranty_SVC,19,138,2019/03/07,NaN,77-649-0500013,NaN
7,CN-Shanghai,PartsRequest-2020110044,JHICC,12106,PM,2020/11/10,2020,3.69e+9,72000401,23.1,...,CNY,1761132.00,不含13%VAT (PO#CAM1443200685),Out of Warranty_SVC,19,138,2019/03/07,NaN,77-649-0500014,NaN
8,CN-Shanghai,PartsRequest-2020110154,YMTC,2263,DOA,2020/11/30,2020,3.69e+9,75016736,1.1,...,NaN,NaN,NaN,Evaluation,57,0,2020/12/08,F1,77-618-02004C1,NaN
9,CN-Shanghai,PartsRequest-2020120074,YMTC,2246,Trouble Shooting,2020/12/10,2020,3.69e+9,73000472,1.1,...,NaN,NaN,NaN,Warranty,36,118,2019/11/11,F1,77-603-0240105,NaN


In [10]:
Part_Desc='PART_DESC'
dft['PART NO_DESC']=dft[Part_Desc].str.strip()
dft['PART NO_DESC']=dft[Part_Desc].str.replace('-',",", regex=True)
dft['PART NO_DESC']=dft[Part_Desc].str.upper()


desc2=dft['PART NO_DESC'].values

X_N=vectorizer.transform(desc2)
predict2=clf2.predict(X_N)
predict2
dft['predict2']=predict2
dft.to_csv('c:\\temp\\predict.csv')

print (predict2)

['other' 'Power Suppl' 'Power Suppl' 'other' 'other' 'other' 'other'
 'other' 'other' 'Column' 'other' 'other' 'other' 'other' 'other'
 'Ion Pump Controller' 'Computer' 'other' 'other' 'other' 'other' 'other'
 'other' 'other' 'other' 'Scroll Pump' 'other' 'other' 'other' 'other'
 'other']
